# DM - cours DATA partie SQL - 2025

The notebook must be returned completed before May 2, 2025 to jean.delpech@adalab.fr

I repeat (again) :
- always build your queries step by step : start small, and make them more and more elaborate
- read the course
- first formulate the operations you want to perform in order, then look for how to implement them in SQL (course, tutorial, doc, stackoverflow, etc.)

You will find 5 `.csv` files in the folder data/DM-SQL. 

## 1. Load them in a database by a method of your choice :

In [1]:
# your code here
import sqlite3
import pandas as pd
import os


csv_folder = "Data"

conn = sqlite3.connect(os.path.join(csv_folder, 'database.db'))

for file in os.listdir(csv_folder):
    if file.endswith(".csv"):
        file_path = os.path.join(csv_folder, file)
        table_name = os.path.splitext(file)[0]  
        df = pd.read_csv(file_path)
        df.to_sql(table_name, conn, if_exists='replace', index=False)
        print(f"{file} importé dans la table {table_name}")

conn.close()

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
units.csv importé dans la table units
admissions.csv importé dans la table admissions
payment.csv importé dans la table payment
patients.csv importé dans la table patients
hospitals.csv importé dans la table hospitals


Then answer the questions :

## 2. List the tables and columns in each table. Create an ERD. Indicate the primary key and any foreign keys for each table. Write down the different relationships and their types (one-to-one, one-to-many, etc.)

In [2]:
# your code here (load the data)
conn = sqlite3.connect("Data/database.db")
cursor = conn.cursor()

In [3]:
# your code here (list tables and columns)
# list all columns of all tables

list_table = 'PRAGMA table_list;'
# first, get table name
cursor.execute(list_table)
for row in cursor.fetchall():
    table = row[1] # to improve readability

    # then get columns names
    column_list = 'PRAGMA table_info(' + table +')'
    print('\n----- table ' + row[1] + ' columns -----\n')
    cursor.execute(column_list)
    
    # print columns names
    for row in  cursor.fetchall():
        column = row[1] # to improve readability
        print(column)
    


----- table hospitals columns -----

id
name
adresse

----- table patients columns -----

id
lastname
firstname
address

----- table payment columns -----

id
admission_id
amount
method

----- table admissions columns -----

id
patient_id
unit_id
entry_date
release_date

----- table units columns -----

id
names
id_hospital
beds

----- table sqlite_schema columns -----

type
name
tbl_name
rootpage
sql

----- table sqlite_temp_schema columns -----

type
name
tbl_name
rootpage
sql


# your code here (code for the ERD)
![Diagramme EDR](edr.png)
```
Table hospitals {
  id INTEGER [pk]
  name TEXT
  address TEXT
}

Table patients {
  id INTEGER [pk]
  lastname TEXT
  firstname TEXT
  address TEXT
}

Table units {
  id INTEGER [pk]
  names TEXT
  id_hospitals INTEGER [ref: > hospitals.id]
  beds INTEGER
}

Table admissions {
  id INTEGER [pk]
  patients_id INTEGER [ref: > patients.id]
  unit_id INTEGER [ref: > units.id]
  entry_date TEXT
  release_date TEXT
}

Table payment {
  id INTEGER [pk]
  admissions_id INTEGER [ref: > admissions.id]
  amount REAL
  method TEXT
} '''

Your responses :
- Relationship 1 : Between units and hospital of type Many-to-One

- Relationship 2 : Between admission and patients of type Many-to-One

- Relationship 3 : Between admission and units of type Many-to-One

- Relationship 4 : Between payment and admission of type Many-to-One or One-to-One

## 3. How many hospitals is there in the database ?

In [24]:
# your code here
cursor.execute("SELECT DISTINCT COUNT(*) FROM hospitals;")

result = cursor.fetchone()
print(f"Il y a {result[0]} hôpitaux dans la base de données.")


Il y a 20 hôpitaux dans la base de données.


## 4.How many hospital units of each type are there, overall (all hospitals combined)?

ex. output:
```
('Cardiologie', 17)
('Gastro-entérologie', 12)
etc.
```

In [25]:
# your code here
cursor.execute("SELECT names, COUNT(*) FROM units GROUP BY names;")

results = cursor.fetchall()

for result in results:
    print(result)


('Cardiologie', 17)
('Gastro-entérologie', 12)
('Néphrologie', 17)
('ORL', 15)
('Oncologie', 12)
('Psychiatrie', 17)


## 5. List the different units by hospitals (hospitals names and units names should both appear in the list) :

ex. output :

```
('Centre Hospitalier Universitaire de Bordeaux', 'Cardiologie')
('Centre Hospitalier Universitaire de Bordeaux', 'Gastro-entérologie')
('Centre Hospitalier Universitaire de Bordeaux', 'ORL')
('Centre Hospitalier Universitaire de Bordeaux', 'Psychiatrie')
('Hôpital Saint-Louis', 'Cardiologie')
('Hôpital Saint-Louis', 'Gastro-entérologie')
etc.
``` 

In [5]:
# your code here
cursor.execute("""
    SELECT hospitals.name, units.names
    FROM units
    JOIN hospitals ON units.id_hospital = hospitals.id
    ORDER BY hospitals.name, units.names;
""")
results = cursor.fetchall()

for result in results:
    print(result)

('Centre Hospitalier Universitaire de Bordeaux', 'Cardiologie')
('Centre Hospitalier Universitaire de Bordeaux', 'Gastro-entérologie')
('Centre Hospitalier Universitaire de Bordeaux', 'ORL')
('Centre Hospitalier Universitaire de Bordeaux', 'Psychiatrie')
('Centre Hospitalier Universitaire de Caen', 'Cardiologie')
('Centre Hospitalier Universitaire de Caen', 'Gastro-entérologie')
('Centre Hospitalier Universitaire de Caen', 'Néphrologie')
('Centre Hospitalier Universitaire de Caen', 'ORL')
('Centre Hospitalier Universitaire de Caen', 'Psychiatrie')
('Centre Hospitalier Universitaire de Clermont-Ferrand', 'Cardiologie')
('Centre Hospitalier Universitaire de Clermont-Ferrand', 'Néphrologie')
('Centre Hospitalier Universitaire de Clermont-Ferrand', 'Oncologie')
('Centre Hospitalier Universitaire de Dijon', 'Néphrologie')
('Centre Hospitalier Universitaire de Dijon', 'ORL')
('Centre Hospitalier Universitaire de Dijon', 'Oncologie')
('Centre Hospitalier Universitaire de Dijon', 'Psychiatrie'

## 6. How many units in each hospital :

ex. output :

```
('Centre Hospitalier Universitaire de Bordeaux', 4)
('Centre Hospitalier Universitaire de Caen', 5)
('Centre Hospitalier Universitaire de Clermont-Ferrand', 3)
etc.
```

In [7]:
# your code here
cursor.execute("""
    SELECT hospitals.name, COUNT(units.id) as unit_count
    FROM hospitals
    LEFT JOIN units ON hospitals.id = units.id_hospital
    GROUP BY hospitals.name
    ORDER BY hospitals.name;
""")
results = cursor.fetchall()

for result in results:
    print(result)

('Centre Hospitalier Universitaire de Bordeaux', 4)
('Centre Hospitalier Universitaire de Caen', 5)
('Centre Hospitalier Universitaire de Clermont-Ferrand', 3)
('Centre Hospitalier Universitaire de Dijon', 4)
('Centre Hospitalier Universitaire de Grenoble', 6)
('Centre Hospitalier Universitaire de Lille', 6)
('Centre Hospitalier Universitaire de Limoges', 6)
('Centre Hospitalier Universitaire de Lyon', 3)
('Centre Hospitalier Universitaire de Montpellier', 3)
('Centre Hospitalier Universitaire de Nantes', 4)
('Centre Hospitalier Universitaire de Nice', 3)
('Centre Hospitalier Universitaire de Rennes', 6)
('Centre Hospitalier Universitaire de Rouen', 5)
('Centre Hospitalier Universitaire de Strasbourg', 6)
('Centre Hospitalier Universitaire de Toulouse', 3)
('Hôpital Bichat-Claude Bernard', 6)
('Hôpital Necker-Enfants Malades', 4)
('Hôpital Saint-Louis', 6)
('Hôpital de la Pitié-Salpêtrière', 4)
('Hôpital de la Timone', 3)


## 7. There are two ways to answer this question: a complex one that yields a bonus, and a longer but easier one (so no bonus). Choose between A or B:

**A.** Write a query that associates each hospital with its department of origin (number):

* You need to extract the first two digits of the postal code (address column)
* Hint 1: To determine the position of the postal code in the address, observe the text and see if a character, always placed in the same location, would allow you to identify the position of the postal code
* Hint 2: The postal code is always 5 characters long, and we're interested in the first two
* Hint 3: You may need to use the same function twice to indicate where the substring begins and ends

Recommendation: When writing a complex query, work your way through it.
* Start by writing a query that lists the hospital addresses
* Then add instructions to this query that remove the beginning of the text up to the postal code
* Then add instructions to display only the portion of text (numbers) you want
* Etc.

**B.** If you can't answer this question the first way (A, which is difficult), add a "department" column by hand to the hospital table (using the appropriate queries of course)

ex. output :

```
('Centre Hospitalier Universitaire de Bordeaux', '33')
('Hôpital Saint-Louis', '75')
('Centre Hospitalier Universitaire de Strasbourg', '67')
('Centre Hospitalier Universitaire de Montpellier', '34')
etc.
```

In [16]:
# your code here
cursor.execute("""
    SELECT
        hospitals.name,
        SUBSTR(
            SUBSTR(
                hospitals.adresse,
                INSTR(hospitals.adresse, ', ') + 2,  
                5                                    
            ),
            1, 2                                   
        ) AS department
    FROM hospitals;
""")
results = cursor.fetchall()

for result in results:
    print(result)

('Centre Hospitalier Universitaire de Bordeaux', '33')
('Hôpital Saint-Louis', '75')
('Centre Hospitalier Universitaire de Strasbourg', '67')
('Centre Hospitalier Universitaire de Montpellier', '34')
('Hôpital Necker-Enfants Malades', '75')
('Centre Hospitalier Universitaire de Toulouse', '31')
('Hôpital de la Timone', '13')
('Centre Hospitalier Universitaire de Nice', '06')
('Centre Hospitalier Universitaire de Nantes', '44')
('Hôpital de la Pitié-Salpêtrière', '75')
('Centre Hospitalier Universitaire de Rennes', '35')
('Centre Hospitalier Universitaire de Lille', '59')
('Centre Hospitalier Universitaire de Grenoble', '38')
('Hôpital Bichat-Claude Bernard', '75')
('Centre Hospitalier Universitaire de Clermont-Ferrand', '63')
('Centre Hospitalier Universitaire de Dijon', '21')
('Centre Hospitalier Universitaire de Lyon', '69')
('Centre Hospitalier Universitaire de Rouen', '76')
('Centre Hospitalier Universitaire de Caen', '14')
('Centre Hospitalier Universitaire de Limoges', '87')


## 8. Are there multiple hospitals in the same departments? List the number of hospitals per department, listed in ascending order by department number:

ex. output :

```
('06', 1)
('13', 1)
('14', 1)
('21', 1)
etc.
```

In [17]:
# your code here
cursor.execute("""
    SELECT 
        SUBSTR(
            SUBSTR(
                hospitals.adresse, 
                INSTR(hospitals.adresse, ', ') + 2,
                5
            ),
            1, 2
        ) AS department,
        COUNT(*) AS hospital_count
    FROM hospitals
    GROUP BY department
    ORDER BY department;
""")
results = cursor.fetchall()

for result in results:
    print(result)

('06', 1)
('13', 1)
('14', 1)
('21', 1)
('31', 1)
('33', 1)
('34', 1)
('35', 1)
('38', 1)
('44', 1)
('59', 1)
('63', 1)
('67', 1)
('69', 1)
('75', 4)
('76', 1)
('87', 1)


## 9. List the number of hospital services in each department, and only display those with more than 4 (order the results by decreasing number of services):

ex. output:

```
('75', 20)
('87', 6)
('67', 6)
etc.
```

In [19]:
# your code here
cursor.execute("""
    SELECT 
        SUBSTR(
            SUBSTR(
                hospitals.adresse, 
                INSTR(hospitals.adresse, ', ') + 2,
                5
            ),
            1, 2
        ) AS department,
        COUNT(*) AS units_count
    FROM units
    JOIN hospitals ON units.id_hospital = hospitals.id
    GROUP BY department
    HAVING COUNT(*) > 4
    ORDER BY units_count DESC;
""")
results = cursor.fetchall()

for result in results:
    print(result)

('75', 20)
('87', 6)
('67', 6)
('59', 6)
('38', 6)
('35', 6)
('76', 5)
('14', 5)


## 10. Admissions have a date of entry into the hospital, and a date of discharge / release. Calculate the average length (duration) of all admissions recorded in the database (in days, rounded to one decimal place).

In [20]:
# your code here
cursor.execute("""
    SELECT ROUND(AVG(JULIANDAY(release_date) - JULIANDAY(entry_date)), 1) AS avg_duration
    FROM admissions
    WHERE release_date IS NOT NULL;
""")
result = cursor.fetchone()
print(f"Durée moyenne des admissions: {result[0]} jours")

Durée moyenne des admissions: 14.6 jours


## 11. Calculate the average length of a cardiology admission (all hospitals combined).

In [21]:
# your code here
cursor.execute("""
    SELECT ROUND(AVG(JULIANDAY(a.release_date) - JULIANDAY(a.entry_date)), 1) AS avg_duration
    FROM admissions a
    JOIN units u ON a.unit_id = u.id
    WHERE u.names LIKE '%Cardiologie%' 
    AND a.release_date IS NOT NULL;
""")
result = cursor.fetchone()
print(f"Durée moyenne en cardiologie: {result[0]} jours")

Durée moyenne en cardiologie: 14.2 jours


## 12. Compare (list) the average length of time for a cardiology admission between all hospitals with a cardiology department. Rank the results in descending order of this average length.

ex. output:

```
('Hôpital de la Timone', 'Cardiologie', 16.5)
('Centre Hospitalier Universitaire de Caen', 'Cardiologie', 16.0)
('Centre Hospitalier Universitaire de Bordeaux', 'Cardiologie', 15.2)
('Centre Hospitalier Universitaire de Strasbourg', 'Cardiologie', 15.2)
etc.
```

In [23]:
# your code here
cursor.execute("""
    SELECT 
        h.name,
        'Cardiologie' AS department,
        ROUND(AVG(JULIANDAY(a.release_date) - JULIANDAY(a.entry_date)), 1) AS avg_duration
    FROM admissions a
    JOIN units u ON a.unit_id = u.id
    JOIN hospitals h ON u.id_hospital = h.id
    WHERE u.names LIKE '%Cardiologie%'
    AND a.release_date IS NOT NULL
    GROUP BY h.name
    ORDER BY avg_duration DESC;
""")
results = cursor.fetchall()

for result in results:
    print(result)

('Hôpital de la Timone', 'Cardiologie', 16.5)
('Centre Hospitalier Universitaire de Caen', 'Cardiologie', 16.0)
('Centre Hospitalier Universitaire de Strasbourg', 'Cardiologie', 15.2)
('Centre Hospitalier Universitaire de Bordeaux', 'Cardiologie', 15.2)
('Hôpital Bichat-Claude Bernard', 'Cardiologie', 14.8)
('Hôpital Saint-Louis', 'Cardiologie', 14.6)
('Centre Hospitalier Universitaire de Rennes', 'Cardiologie', 14.6)
('Hôpital de la Pitié-Salpêtrière', 'Cardiologie', 14.5)
('Centre Hospitalier Universitaire de Lille', 'Cardiologie', 14.4)
('Centre Hospitalier Universitaire de Limoges', 'Cardiologie', 14.1)
('Centre Hospitalier Universitaire de Lyon', 'Cardiologie', 14.0)
('Centre Hospitalier Universitaire de Montpellier', 'Cardiologie', 13.6)
('Centre Hospitalier Universitaire de Rouen', 'Cardiologie', 13.2)
('Centre Hospitalier Universitaire de Toulouse', 'Cardiologie', 12.6)
('Centre Hospitalier Universitaire de Nantes', 'Cardiologie', 12.2)
('Centre Hospitalier Universitaire de Gre

## 13. What query displays the 25% of hospitals with the shortest hospital stays for cardiology? Present the output as follows:

```
('Cardiologie', 'Centre Hospitalier Universitaire de Clermont-Ferrand', '<25%')
('Cardiologie', 'Centre Hospitalier Universitaire de Grenoble', '<25%')
('Cardiologie', 'Centre Hospitalier Universitaire de Nantes', '<25%')
('Cardiologie', 'Centre Hospitalier Universitaire de Toulouse', '<25%')
('Cardiologie', 'Centre Hospitalier Universitaire de Rouen', '<25%')
etc.
```



In [26]:
# your code here
cursor.execute("""
    SELECT 
        'Cardiologie' AS department,
        h.name AS hospital_name,
        '<25%' AS percentile
    FROM admissions a
    JOIN units u ON a.unit_id = u.id
    JOIN hospitals h ON u.id_hospital = h.id
    WHERE u.names LIKE '%Cardiologie%'
    AND a.release_date IS NOT NULL
    GROUP BY h.name
    ORDER BY AVG(JULIANDAY(a.release_date) - JULIANDAY(a.entry_date))
    LIMIT (SELECT COUNT(DISTINCT h2.id) / 4 
          FROM hospitals h2
          JOIN units u2 ON u2.id_hospital = h2.id
          WHERE u2.names LIKE '%Cardiologie%');
""")
results = cursor.fetchall()

for result in results:
    print(result)

('Cardiologie', 'Centre Hospitalier Universitaire de Clermont-Ferrand', '<25%')
('Cardiologie', 'Centre Hospitalier Universitaire de Grenoble', '<25%')
('Cardiologie', 'Centre Hospitalier Universitaire de Nantes', '<25%')
('Cardiologie', 'Centre Hospitalier Universitaire de Toulouse', '<25%')
